*Give credit for code foundation here*

### Grab vocab list from Github
This can be changed at a later time. All that matter is that the vocab names are directly from QuickDraw and that the size of the output layer reflects the number of items in the text file.

In [ ]:
!wget https://raw.githubusercontent.com/Capstone-Projects-2023-Fall/project-smartspeech/SS-121-122/backend/model/quickdraw-vocab.txt

--2023-12-10 19:56:50--  https://raw.githubusercontent.com/Capstone-Projects-2023-Fall/project-smartspeech/SS-121-122/backend/model/quickdraw-vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391 [text/plain]
Saving to: ‘quickdraw-vocab.txt’

quickdraw-vocab.txt 100%[===================>]     391  --.-KB/s    in 0s      

2023-12-10 19:56:51 (9.39 MB/s) - ‘quickdraw-vocab.txt’ saved [391/391]



### Store class names into a list
Remove newline characters and replace all spaces with underscores so that the data is equally represented.

In [ ]:
f = open('quickdraw-vocab.txt', 'r')
classes = f.readlines()
f.close()

In [ ]:
classes = [c.replace('\n', '').replace(' ', '_') for c in classes]

### Download data from Google


In [ ]:
!mkdir data

In [ ]:
import urllib.request
def download():
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    cls_url = c.replace('_', '%20')
    path = base+cls_url+'.npy'
    print(path)
    urllib.request.urlretrieve(path, 'data/'+c+'.npy')

In [ ]:
download()

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/airplane.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bandage.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bee.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bicycle.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/blueberry.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bowtie.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/butterfly.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/camera.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/car.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/carrot.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/chair.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/circle.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_b

### Imports

In [ ]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from tensorflow import keras

### Load data

In [25]:
def load_data(root, vfold_ratio=0.2, max_items_per_class= 4000):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    #load each data file
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None

    #randomize the dataset
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    #separate into training and testing
    vfold_size = int(x.shape[0]/100*(vfold_ratio*100))

    x_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    x_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return x_train, y_train, x_test, y_test, class_names

In [26]:
x_train, y_train, x_test, y_test, class_names = load_data('data')
num_classes = len(class_names)
image_size = 28

In [27]:
print(len(x_train))

172800


### Preprocess data

In [28]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Define model

In [31]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(54, activation='softmax'))
# Train model
adam = tf.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 14, 14, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 7, 7, 32)          0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 3, 3, 64)         

### Train model

In [32]:
model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

Epoch 1/5
608/608 - 6s - loss: 1.4308 - top_k_categorical_accuracy: 0.8534 - val_loss: 0.9240 - val_top_k_categorical_accuracy: 0.9264 - 6s/epoch - 10ms/step
Epoch 2/5
608/608 - 4s - loss: 0.8041 - top_k_categorical_accuracy: 0.9375 - val_loss: 0.7644 - val_top_k_categorical_accuracy: 0.9387 - 4s/epoch - 6ms/step
Epoch 3/5
608/608 - 3s - loss: 0.6834 - top_k_categorical_accuracy: 0.9477 - val_loss: 0.6962 - val_top_k_categorical_accuracy: 0.9456 - 3s/epoch - 5ms/step
Epoch 4/5
608/608 - 3s - loss: 0.6147 - top_k_categorical_accuracy: 0.9533 - val_loss: 0.6596 - val_top_k_categorical_accuracy: 0.9472 - 3s/epoch - 6ms/step
Epoch 5/5
608/608 - 4s - loss: 0.5685 - top_k_categorical_accuracy: 0.9571 - val_loss: 0.6336 - val_top_k_categorical_accuracy: 0.9491 - 4s/epoch - 6ms/step


### Test model

In [33]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Test accuarcy: 95.30%


### Store classes

In [34]:
with open('class_names.txt', 'w') as file_handler:
    for item in class_names:
        file_handler.write("{}\n".format(item))

### Export weights and relevant files

In [35]:
!pip install tensorflowjs

ERROR: Operation cancelled by user


In [36]:
model.save('keras.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
!mkdir model
!tensorflowjs_converter --input_format keras keras.h5 model/

mkdir: cannot create directory ‘model’: File exists
2023-12-10 20:35:57.797800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 20:35:57.797905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 20:35:57.961676: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 20:36:00.644742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [38]:

!cp class_names.txt model/class_names.txt

In [39]:
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/group1-shard1of1.bin (deflated 7%)
  adding: model/.ipynb_checkpoints/ (stored 0%)
  adding: model/model.json (deflated 82%)
  adding: model/class_names.txt (deflated 38%)


In [40]:
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>